# Análise Exploratória dos Dados - Ureia
---

In [ ]:
import os
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import calendar

from pathlib import Path
from typing import List
from statsmodels.tsa.seasonal import STL

print("Todas as bibliotecas necessárias foram importadas com sucesso.")

## Configs and Paths:
---

Adicionando todos os caminhos que serão utilizados durante as análises.

In [ ]:
DATA_DIR = os.path.abspath("/lakehouse/default/Files/dados_estruturados")
DATA_DIR = Path(DATA_DIR)
RAW_DIR = DATA_DIR / "raw"

paths = {
    "merged": RAW_DIR / "merged_data.csv",
    "pink": RAW_DIR / "pink_sheet_monthly.csv",
    "ptax": RAW_DIR / "bcb_ptax_usdbrl_1990-01_2025-12.csv",
    "gscpi": RAW_DIR / "nyfed_gscpi.csv",
    "oni": RAW_DIR / "noaa_oni.csv",
    "gpr": RAW_DIR / "gpr.csv",
    "events": RAW_DIR / "main_events.csv",
    "trade": RAW_DIR / "urea_trade_features.csv",
    "india": RAW_DIR / "india_urea_hs6_by_partner_wits.csv",
}

UREA_COL = "urea_usd"

## Métodos auxiliares:
---

In [ ]:
def to_month_start(dt_series: pd.Series) -> pd.Series:
    s = pd.to_datetime(dt_series, errors="coerce")
    return s.dt.to_period("M").dt.to_timestamp()

def add_seasonality(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    m = out.index.month.astype(int)
    out["month"] = m
    out["month_sin"] = np.sin(2 * np.pi * m / 12)
    out["month_cos"] = np.cos(2 * np.pi * m / 12)
    return out


def add_lags(df: pd.DataFrame, cols: List[str], lags: List[int]) -> pd.DataFrame:
    out = df.copy()
    new_cols = {}
    for c in cols:
        if c not in out.columns:
            continue
        for L in lags:
            new_cols[f"{c}_lag{L}"] = out[c].shift(L)
    if new_cols:
        out = pd.concat([out, pd.DataFrame(new_cols)], axis=1)
    return out


def add_rolling(df: pd.DataFrame, cols: List[str], windows: List[int]) -> pd.DataFrame:
    out = df.copy()
    new_cols = {}
    for c in cols:
        if c not in out.columns:
            continue
        for w in windows:
            new_cols[f"{c}_ma{w}"] = out[c].rolling(w).mean()
    if new_cols:
        out = pd.concat([out, pd.DataFrame(new_cols)], axis=1)
    return out


def lag_correlation_table(df: pd.DataFrame, target: str, features: List[str], lags: List[int]) -> pd.DataFrame:
    rows = []
    y = df[target]
    for f in features:
        if f not in df.columns:
            continue
        for L in lags:
            corr = y.corr(df[f].shift(L))
            rows.append({"feature": f, "lag": L, "corr": corr})
    out = pd.DataFrame(rows).dropna()
    out["abs_corr"] = out["corr"].abs()
    return out.sort_values(["abs_corr"], ascending=False).reset_index(drop=True)

def plot_series(df: pd.DataFrame, cols: List[str], outpath: str, title: str) -> None:
    plt.figure(figsize=(12, 5))
    for c in cols:
        if c in df.columns:
            plt.plot(df.index, df[c], label=c)
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.close()


def plot_bar(df: pd.DataFrame, x: str, y: str, outpath: str, title: str, top_n: int = 20) -> None:
    d = df.head(top_n).copy()
    plt.figure(figsize=(10, 6))
    plt.barh(d[x][::-1], d[y][::-1])
    plt.title(title)
    plt.tight_layout()
    plt.close()

def dataset_overview(name: str, df: pd.DataFrame, date_col: str = None):
    out = {
        "dataset": name,
        "rows": len(df),
        "cols": df.shape[1],
    }
    if date_col and date_col in df.columns:
        d = pd.to_datetime(df[date_col], errors="coerce")
        out["min_date"] = d.min()
        out["max_date"] = d.max()
    else:
        out["min_date"] = pd.NaT
        out["max_date"] = pd.NaT
    out["missing_cells_pct"] = round(float(df.isna().mean().mean() * 100), 3)
    return out


def zscore(s: pd.Series) -> pd.Series:
    s = s.astype(float)
    return (s - s.mean()) / s.std(ddof=0)

## Carregando os datasets:
---
Atribuindo cada dataset a uma variável para facilitar manipulações.

In [ ]:
df_merged = pd.read_csv(paths["merged"])
df_merged["date"] = pd.to_datetime(df_merged["date"], errors="coerce")
df_merged = df_merged.sort_values("date").reset_index(drop=True)

df_pink = pd.read_csv(paths["pink"])
df_pink["date"] = pd.to_datetime(df_pink["date"], errors="coerce")

df_ptax = pd.read_csv(paths["ptax"])
df_ptax["date"] = pd.to_datetime(df_ptax["date"], errors="coerce")

df_gscpi = pd.read_csv(paths["gscpi"])
df_gscpi["date"] = pd.to_datetime(df_gscpi["date"], errors="coerce")

df_oni = pd.read_csv(paths["oni"])
df_oni["date"] = pd.to_datetime(df_oni["date"], errors="coerce")

df_gpr = pd.read_csv(paths["gpr"])
df_gpr["date"] = pd.to_datetime(df_gpr["date"], errors="coerce")

df_events = pd.read_csv(paths["events"])
df_events["date"] = pd.to_datetime(df_events["date"], errors="coerce")

df_trade = pd.read_csv(paths["trade"])
df_trade["date"] = pd.to_datetime(df_trade["date"], errors="coerce")
mask = df_trade["flowDesc"] == "Re-import"
df_trade = df_trade[~mask].copy()

df_trade_pivoted = df_trade.pivot_table(
    index='date',
    columns='flowDesc',
    values=['CHN_tonnes', 'CHN_value_usd', 'BRA_tonnes', 'BRA_value_usd'],
    aggfunc='first'
)
df_trade_pivoted.columns = ['_'.join(col).strip() for col in df_trade_pivoted.columns.values]

df_india = pd.read_csv(paths["india"])
df_india["date"] = pd.to_datetime(df_india["year"].astype(str) + "-01-01", errors="coerce")

df_merged = df_merged.set_index("date")
df_merged = df_merged.drop(columns=["index"])

if UREA_COL not in df_merged.columns:
    raise ValueError(f"Coluna-alvo não encontrada: {UREA_COL}")

# Adiciona features de comércio internacional ao dataset principal
df_merged = df_merged.merge(df_trade_pivoted, left_index=True, right_index=True, how="left")
df_merged['urea_brl'] = df_merged[UREA_COL] * df_merged['usdbrl']

df_merged.to_csv(RAW_DIR / "merged_data_with_trade.csv")


# Aplica filtro de data inicial como 1995-01-01.
df_merged = df_merged[df_merged.index >= pd.to_datetime("1995-01-01")]


### Gerar overview dos datasets:
---
Permite termos uma noção inicial da sanidade dos dados, verificando sua completude.

In [ ]:
overview = pd.DataFrame([
    dataset_overview("merged_data.csv", df_merged.reset_index(), "date"),
    dataset_overview("pink_sheet_monthly.csv", df_pink, "date"),
    dataset_overview("ptax_usdbrl.csv", df_ptax, "date"),
    dataset_overview("nyfed_gscpi.csv", df_gscpi, "date"),
    dataset_overview("noaa_oni.csv", df_oni, "date"),
    dataset_overview("gpr.csv", df_gpr, "date"),
    dataset_overview("main_events.csv", df_events, "date"),
    dataset_overview("urea_trade_features.csv", df_trade, "date"),
    dataset_overview("india_urea_hs6_by_partner_wits.csv", df_india, "date"),
])

overview

## Análise de Entropia
---
Análise de Entropia para entendermos o quão caótica são as variáveis numéricas que estaremos analisando durante esse estudo.

In [ ]:
from scipy.stats import entropy

# Selecionar as features de interesse
features = ["urea_usd", "crude_oil_usd", "natural_gas_usd", "usdbrl", "gscpi", "gpr"]
df_features = df_merged[features].dropna()

# Calcular a entropia para cada feature
entropy_results = {}

for col in features:
    # Normalizar os dados para criar uma distribuição de probabilidade
    # Usando histograma para discretizar valores contínuos
    hist, bin_edges = np.histogram(df_features[col], bins=30, density=True)
    
    # Normalizar para soma = 1 (distribuição de probabilidade)
    hist = hist / hist.sum()
    
    # Remover bins vazios para evitar log(0)
    hist = hist[hist > 0]
    
    # Calcular entropia
    ent = entropy(hist, base=2)  # base=2 para bits, base=e para nats
    entropy_results[col] = ent

# Criar DataFrame com resultados
entropy_df = pd.DataFrame.from_dict(
    entropy_results, 
    orient='index', 
    columns=['entropy']
).sort_values('entropy', ascending=False)

print("=" * 60)
print("ENTROPIA DAS FEATURES (em bits)")
print("=" * 60)
print(entropy_df)
print("\nInterpretação:")
print("- Maior entropia = maior incerteza/variabilidade")
print("- Menor entropia = mais previsível/concentrada")

# Visualização
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras da entropia
ax1.barh(entropy_df.index, entropy_df['entropy'], color='steelblue')
ax1.set_xlabel('Entropia (bits)', fontsize=11)
ax1.set_title('Entropia por Feature\n(Medida de incerteza/variabilidade)', fontsize=12, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# Distribuições normalizadas para comparação visual
for i, col in enumerate(features):
    hist, bin_edges = np.histogram(df_features[col], bins=30, density=True)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    ax2.plot(bin_centers, hist / hist.sum(), label=f"{col} (H={entropy_results[col]:.2f})", alpha=0.7)

ax2.set_xlabel('Valor normalizado', fontsize=10)
ax2.set_ylabel('Densidade de probabilidade', fontsize=10)
ax2.set_title('Distribuições de Probabilidade das Features', fontsize=12, fontweight='bold')
ax2.legend(fontsize=8, loc='best')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
from scipy.stats import entropy

# Selecionar as features de interesse
features = ["urea_usd", "crude_oil_usd", "natural_gas_usd", "usdbrl", "gscpi", "gpr"]
df_features = df_merged[features].dropna()

# Calcular o Índice de Shannon (Entropia de Shannon) para cada feature
shannon_results = {}

for col in features:
    # Normalizar os dados para criar uma distribuição de probabilidade
    # Usando histograma para discretizar valores contínuos
    hist, bin_edges = np.histogram(df_features[col], bins=30, density=True)
    
    # Normalizar para soma = 1 (distribuição de probabilidade)
    hist = hist / hist.sum()
    
    # Remover bins vazios para evitar log(0)
    hist = hist[hist > 0]
    
    # Calcular Índice de Shannon (entropia em nats - base natural)
    shannon_index = entropy(hist, base=np.e)  # base=e para nats (índice de Shannon clássico)
    shannon_results[col] = shannon_index

# Criar DataFrame com resultados
shannon_df = pd.DataFrame.from_dict(
    shannon_results, 
    orient='index', 
    columns=['shannon_index']
).sort_values('shannon_index', ascending=False)

print("=" * 70)
print("ÍNDICE DE SHANNON (ENTROPIA DE SHANNON) - em nats")
print("=" * 70)
print(shannon_df)
print("\nInterpretação do Índice de Shannon:")
print("- Maior valor = maior diversidade/incerteza na distribuição")
print("- Menor valor = maior concentração/previsibilidade")
print("- Unidade: nats (logaritmo natural)")
print("- Conversão: 1 nat ≈ 1.443 bits")

# Adicionar versão em bits para comparação
shannon_df['shannon_bits'] = shannon_df['shannon_index'] / np.log(2)

print("\n" + "=" * 70)
print("ÍNDICE DE SHANNON - Comparação (nats vs bits)")
print("=" * 70)
print(shannon_df)

# Visualização
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# 1. Gráfico de barras horizontal - Índice de Shannon (nats)
ax1.barh(shannon_df.index, shannon_df['shannon_index'], color='steelblue', alpha=0.8)
ax1.set_xlabel('Índice de Shannon (nats)', fontsize=11)
ax1.set_title('Índice de Shannon por Feature\n(Medida de diversidade/entropia)', 
              fontsize=12, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# 2. Gráfico de barras horizontal - Índice de Shannon (bits)
ax2.barh(shannon_df.index, shannon_df['shannon_bits'], color='coral', alpha=0.8)
ax2.set_xlabel('Índice de Shannon (bits)', fontsize=11)
ax2.set_title('Índice de Shannon por Feature\n(Convertido para bits)', 
              fontsize=12, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

# 3. Distribuições de probabilidade (densidade)
for i, col in enumerate(features):
    hist, bin_edges = np.histogram(df_features[col], bins=30, density=True)
    hist_prob = hist / hist.sum()
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    ax3.plot(bin_centers, hist_prob, 
             label=f"{col}\n(H={shannon_results[col]:.3f} nats)", 
             alpha=0.7, linewidth=2)

ax3.set_xlabel('Valor (normalizado)', fontsize=10)
ax3.set_ylabel('Densidade de probabilidade', fontsize=10)
ax3.set_title('Distribuições de Probabilidade das Features\n(base para cálculo do Índice de Shannon)', 
              fontsize=12, fontweight='bold')
ax3.legend(fontsize=8, loc='best')
ax3.grid(alpha=0.3)

# 4. Comparação visual: nats vs bits
x_pos = np.arange(len(shannon_df))
width = 0.35

ax4.bar(x_pos - width/2, shannon_df['shannon_index'], width, 
        label='Shannon (nats)', color='steelblue', alpha=0.8)
ax4.bar(x_pos + width/2, shannon_df['shannon_bits'], width, 
        label='Shannon (bits)', color='coral', alpha=0.8)

ax4.set_xlabel('Features', fontsize=11)
ax4.set_ylabel('Índice de Shannon', fontsize=11)
ax4.set_title('Comparação: Índice de Shannon em nats vs bits', 
              fontsize=12, fontweight='bold')
ax4.set_xticks(x_pos)
ax4.set_xticklabels(shannon_df.index, rotation=45, ha='right')
ax4.legend(fontsize=10)
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Análise estatística adicional
print("\n" + "=" * 70)
print("ANÁLISE COMPARATIVA - ÍNDICE DE SHANNON")
print("=" * 70)
print(f"\nFeature mais diversa (maior entropia):")
print(f"  → {shannon_df.index[0]}: {shannon_df.iloc[0]['shannon_index']:.4f} nats")
print(f"\nFeature mais concentrada (menor entropia):")
print(f"  → {shannon_df.index[-1]}: {shannon_df.iloc[-1]['shannon_index']:.4f} nats")
print(f"\nRazão max/min: {shannon_df.iloc[0]['shannon_index'] / shannon_df.iloc[-1]['shannon_index']:.2f}x")
print("=" * 70)

## Cobertura temporal por dataset:
---
Deixar explícito quais fontes cobrem quais períodos, verificando se existe sobreposição suficiente para análises conjuntas. É um slide ótimo para justificar por que alguns modelos/insights só fazem sentido a partir de certo ano (por exemplo, se um índice começa muito depois).

In [ ]:
fig = plt.figure(figsize=(10, 4.8))
ax = fig.add_subplot(111)

timeline = overview.dropna(subset=["min_date", "max_date"]).copy()
timeline = timeline.sort_values("min_date")
y = np.arange(len(timeline))
ax.hlines(y=y, xmin=timeline["min_date"], xmax=timeline["max_date"], label="Janela temporal", color="darkblue")
ax.plot(timeline["min_date"], y, marker="o", linestyle="None", label="Início")
ax.plot(timeline["max_date"], y, marker="o", linestyle="None", label="Fim")
ax.vlines(pd.Timestamp("1994-01-01"), ymin=0, ymax=len(timeline)-1, color="red", linestyle="--", label="Janela temporal de análise")
ax.set_yticks(y)
ax.set_yticklabels(timeline["dataset"])
ax.set_title("Cobertura temporal por dataset (início e fim)")
ax.set_xlabel("Data")
ax.legend()

plt.show()

## Mapa de dados faltantes:
---
Serve para enxergar rapidamente se os faltantes estão concentrados em determinados períodos (ex.: início da série) ou em variáveis específicas. Ajuda em decisões como recorte temporal para a análise, escolha de imputação, ou até a exclusão de variáveis pouco confiáveis.

In [ ]:
num_cols = df_merged.select_dtypes(include=[np.number]).columns.tolist()
heat = df_merged[num_cols].isna().astype(int).to_numpy().T

fig, ax = plt.subplots(figsize=(12, 5.2))
im = ax.imshow(heat, aspect="auto", interpolation="nearest", cmap="binary", vmin=0, vmax=1)
ax.set_title("Mapa de dados faltantes (0 = presente, 1 = ausente) - variáveis numéricas")
ax.set_yticks(np.arange(len(num_cols)))
ax.set_yticklabels(num_cols, fontsize=7)

dates = df_merged.index.to_numpy()
if len(dates) > 0:
    years = pd.DatetimeIndex(dates).year
    tick_idx = np.where((pd.DatetimeIndex(dates).month == 1))[0]
    ax.set_xticks(tick_idx[::2] if len(tick_idx) > 25 else tick_idx)
    ax.set_xticklabels(pd.DatetimeIndex(dates)[ax.get_xticks()].year, rotation=90, fontsize=7)

cbar = fig.colorbar(im, ax=ax, fraction=0.02, pad=0.02, ticks=[0, 1])
cbar.set_label("Dado Faltante", rotation=270, labelpad=15)

## Análise de Preço x Tempo (USD e BRL)
---
Verificando como fica a evolução do preço da ureia ao longo dos anos.

In [ ]:
import matplotlib.dates as mdates

feature1 = UREA_COL
feature2 = 'urea_brl'

tmp_df = df_merged[df_merged.index >= pd.to_datetime("1995-01-01")].copy()

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(12, 8))
if feature1 in tmp_df.columns:
    ax1.plot(tmp_df.index, tmp_df[feature1], color='C0', label=feature1)
    ax1.plot(tmp_df.index, tmp_df[feature1].rolling(12, min_periods=6).mean().values, color='gray', label="Média móvel 12m", alpha=0.7)
    ax1.set_ylabel(feature1)
    ax1.legend(loc='upper left')
if feature2 in tmp_df.columns:
    ax2.plot(tmp_df.index, tmp_df[feature2], color='C1', label=feature2)
    ax2.plot(tmp_df.index, tmp_df[feature2].rolling(12, min_periods=6).mean().values, color='gray', label="Média móvel 12m", alpha=0.7)
    ax2.set_ylabel(feature2)
    ax2.legend(loc='upper left')
if feature1 in tmp_df.columns and feature2 in tmp_df.columns:
    ax3.plot(tmp_df.index, tmp_df[feature1], color='C0', label=feature1)
    ax3.plot(tmp_df.index, tmp_df[feature2], color='C1', label=feature2)
    ax3.legend(loc='upper left')
    ax3.set_ylabel('preço/t')

    years = pd.date_range(tmp_df.index.values.min(), tmp_df.index.values.max(), freq="YS")
    ax3.set_xticks(years)
    ax3.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
    ax3.tick_params(axis="x", rotation=60)

ax2.set_xlabel('Date')
fig.suptitle(f'Ureia em dólares e em reais ao longo do tempo (30 anos)', fontsize=16)
fig.tight_layout(rect=[0, 0.03, 1, 0.99])
plt.show()

In [ ]:
import matplotlib.dates as mdates

feature1 = 'urea_usd'
feature2 = 'urea_brl'

tmp_df = df_merged[df_merged.index >= pd.to_datetime("1995-01-01")].copy()

# Normalizar com z-score
tmp_df['urea_usd_norm'] = (tmp_df[feature1] - tmp_df[feature1].mean()) / tmp_df[feature1].std()
tmp_df['urea_brl_norm'] = (tmp_df[feature2] - tmp_df[feature2].mean()) / tmp_df[feature2].std()

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(12, 8))

# Gráfico 1: Séries normalizadas
ax1.plot(tmp_df.index, tmp_df['urea_usd_norm'], color='C0', label='urea_usd (normalizado)', linewidth=2)
ax1.plot(tmp_df.index, tmp_df['urea_brl_norm'], color='C1', label='urea_brl (normalizado)', linewidth=2)
ax1.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax1.set_ylabel('Z-score')
ax1.set_title('Ureia USD vs BRL - Séries Normalizadas (Z-score)')
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)

# Gráfico 2: Escala original (para referência)
ax2_right = ax2.twinx()
ax2.plot(tmp_df.index, tmp_df[feature1], color='C0', label='urea_usd', linewidth=2)
ax2_right.plot(tmp_df.index, tmp_df[feature2], color='C1', label='urea_brl', linewidth=2)
ax2.set_ylabel('US$/t', color='C0')
ax2_right.set_ylabel('R$/t', color='C1')
ax2.tick_params(axis='y', labelcolor='C0')
ax2_right.tick_params(axis='y', labelcolor='C1')
ax2.set_title('Ureia USD vs BRL - Escala Original')
ax2.legend(loc='upper left', fontsize=8)
ax2_right.legend(loc='upper right', fontsize=8)
ax2.grid(True, alpha=0.3)

years = pd.date_range(tmp_df.index.values.min(), tmp_df.index.values.max(), freq="YS")
ax2.set_xticks(years)
ax2.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
ax2.tick_params(axis="x", rotation=60)

fig.tight_layout()
plt.show()

### Variação Mensal:
---

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(tmp_df.index, tmp_df[feature1].pct_change() * 100, color='C0', label='urea_usd (% change)', linewidth=1.5, alpha=0.8)
ax.plot(tmp_df.index, tmp_df[feature2].pct_change() * 100, color='C1', label='urea_brl (% change)', linewidth=1.5, alpha=0.8)
ax.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax.set_ylabel('Variação (%)')
ax.set_title('Ureia USD vs BRL - Variação Mensal (%)')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
s_urea = df_merged[UREA_COL].astype(float)

ret = s_urea.pct_change() * 100
vol12 = ret.rolling(12, min_periods=6).std()

fig = plt.figure(figsize=(12, 4.2))
ax = fig.add_subplot(111)
ax.plot(ret.index, ret.values, label="Variação % mensal")
ax.plot(vol12.index, vol12.values, label="Volatilidade 12m (desvio-padrão)")
ax.set_title("Ureia (USD) - variação percentual mensal x volatilidade móvel")
ax.set_ylabel("%")
ax.legend(loc="best")

plt.show()

## Análises de Boxplot:
---

In [ ]:
s_urea = s_urea[s_urea.index >= pd.to_datetime("1995-01-01")]

tmp = pd.DataFrame({"date": s_urea.index, "urea": s_urea.values})
tmp["month"] = tmp["date"].dt.month
data_by_month = [tmp.loc[tmp["month"] == m, "urea"].dropna().values for m in range(1, 13)]

fig = plt.figure(figsize=(10.8, 4.2))
ax = fig.add_subplot(111)
ax.boxplot(data_by_month, labels=[str(m) for m in range(1, 13)], showfliers=False)
ax.set_title("Boxplot - distribuição do preço da Ureia por mês (30 anos - 1995 a 2025)")
ax.set_xlabel("Mês")
ax.set_ylabel("US$/t")

plt.show()

In [ ]:
tmp = pd.DataFrame({"date": s_urea.index, "urea": s_urea.values})
tmp["month"] = tmp["date"].dt.month
tmp["year"] = tmp["date"].dt.year
tmp["year_group"] = (tmp["year"] // 10) * 10

year_groups = sorted(tmp["year_group"].unique())
year_labels = [f"{yr}-{yr+9}" if yr + 9 <= tmp["year"].max() else f"{yr}-{tmp['year'].max()}" for yr in year_groups]

fig, ax = plt.subplots(figsize=(16, 6))

# Posições para os boxplots (mês, depois período)
positions = []
data_to_plot = []
colors_list = []
month_to_positions = {m: [] for m in range(1, 13)}  # Mapear mês para posições
pos = 1

color_palette = ['lightblue', 'lightgreen', 'lightpink']

for month in range(1, 13):
    for idx, year_group in enumerate(year_groups):
        data = tmp.loc[(tmp["year_group"] == year_group) & (tmp["month"] == month), "urea"].dropna().values
        if len(data) > 0:
            data_to_plot.append(data)
            positions.append(pos)
            colors_list.append(color_palette[idx % len(color_palette)])
            month_to_positions[month].append(pos)  # Guardar posição para este mês
            pos += 1
    pos += 1  # Espaço entre meses

bp = ax.boxplot(data_to_plot, positions=positions, widths=0.6, patch_artist=True, showfliers=False)

# Colorir os boxplots
for patch, color in zip(bp['boxes'], colors_list):
    patch.set_facecolor(color)

# Adicionar labels para os meses
month_positions = []
month_labels = []
for month in range(1, 13):
    if month_to_positions[month]:  # Se há dados para este mês
        avg_pos = np.mean(month_to_positions[month])
        month_positions.append(avg_pos)
        month_labels.append(str(month))

ax.set_xticks(month_positions)
ax.set_xticklabels(month_labels)
ax.set_ylabel("US$/t")
ax.set_title("Boxplot - Distribuição do preço da Ureia por mês e período de 10 anos", fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# Legenda com cores para períodos
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=color_palette[i % len(color_palette)], label=year_labels[i]) 
                   for i in range(len(year_groups))]
ax.legend(handles=legend_elements, loc='upper left', title='Período', fontsize=9)

plt.tight_layout()
plt.show()

## Análise STL (Season-Trend with LOESS):
---

In [ ]:
stl_series = s_urea.dropna()
stl = STL(stl_series, period=12, robust=True).fit()

translations = {
    "trend": "tendência",
    "seasonal": "sazonalidade",
    "resid": "residual",
}

for comp_name, comp_series in [
    ("trend", stl.trend),
    ("seasonal", stl.seasonal),
    ("resid", stl.resid),
]:
    fig, ax = plt.subplots(figsize=(12, 4.0))
    ax.plot(comp_series.index, comp_series.values, linewidth=1.5)
    ax.set_title(f"Decomposição STL - componente: {translations[comp_name]}", fontsize=13, fontweight='bold')
    ax.set_ylabel("US$/t", fontsize=11)
    ax.set_xlabel("Ano", fontsize=11)
    
    # Configurar anos no eixo X
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    
    # Rotação e alinhamento
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right', fontsize=9)
    
    # Grid para melhor leitura
    ax.grid(True, alpha=0.3, linestyle='--')
    
    fig.tight_layout()

plt.show()

In [ ]:
seasonal = stl.seasonal

data_corte = seasonal.index.max() - pd.DateOffset(years=5)
seasonal_recent = seasonal[seasonal.index > data_corte]
monthly_avg = seasonal_recent.groupby(seasonal_recent.index.month).mean()

# Criar nomes dos meses para o eixo X
month_names = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 
               'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']

# Plotar o Gráfico de Barras
fig, ax = plt.subplots(figsize=(10, 5))

# Criar cores condicionais: Verde para preço baixo (compra), Vermelho para preço alto
colors = ['green' if x < 0 else 'red' for x in monthly_avg.values]

bars = ax.bar(month_names, monthly_avg.values, color=colors, alpha=0.7, edgecolor='black')

# Estilização
ax.set_title("Sazonalidade média mensal (últimos 5 anos)", fontsize=14)
ax.set_ylabel("Impacto no Preço (US$/t)", fontsize=12)
ax.set_xlabel("Mês", fontsize=12)
ax.axhline(0, color='black', linewidth=1, linestyle='-') # Linha zero
ax.grid(axis='y', linestyle='--', alpha=0.5)

# Adicionar o valor exato em cima/baixo das barras
for bar in bars:
    height = bar.get_height()
    offset = 2 if height > 0 else -2 
    ax.text(bar.get_x() + bar.get_width()/2., height + offset,
            f'{height:.1f}',
            ha='center', va='bottom' if height > 0 else 'top', 
            fontsize=10, fontweight='bold', color='black')

ax.set_ylim(-60, 60)

plt.tight_layout()
plt.show()

## Correlações:
---

In [ ]:
import seaborn as sns
from scipy.stats import pearsonr
from statsmodels.tsa.stattools import ccf

# ============================================================================
# 1. HEATMAP DE CORRELAÇÃO - NÍVEIS
# ============================================================================
print("="*70)
print("1. MATRIZ DE CORRELAÇÃO - NÍVEIS")
print("="*70)

key_vars = ['urea_usd', 'natural_gas_usd', 'crude_oil_usd', 'usdbrl',
            'gscpi', 'oni', 'gpr', 'dap_usd', 'potassium_usd', 'maize_usd', 'wheat_usd', 'soybeans_usd']
key_vars = [v for v in key_vars if v in df_merged.columns]

corr_levels = df_merged[key_vars].corr()

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(corr_levels, annot=True, fmt='.3f', cmap='RdYlGn', center=0,
            vmin=-1, vmax=1, square=True, linewidths=0.5, 
            cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Matriz de Correlação - Níveis (Pearson)', 
             fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# ============================================================================
# 2. HEATMAP DE CORRELAÇÃO - VARIAÇÕES MOM (%)
# ============================================================================
print("\n" + "="*70)
print("2. MATRIZ DE CORRELAÇÃO - VARIAÇÕES MOM (%)")
print("="*70)

df_mom = df_merged[key_vars].pct_change() * 100
df_mom.columns = [f"{c}_mom" for c in df_mom.columns]
corr_mom = df_mom.corr()

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(corr_mom, annot=True, fmt='.3f', cmap='RdYlGn', center=0,
            vmin=-1, vmax=1, square=True, linewidths=0.5,
            cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Matriz de Correlação - Variações MoM (%)', 
             fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# ============================================================================
# 3. HEATMAP DE CORRELAÇÃO - VARIAÇÕES YOY (%)
# ============================================================================
print("\n" + "="*70)
print("3. MATRIZ DE CORRELAÇÃO - VARIAÇÕES YOY (%)")
print("="*70)

df_yoy = df_merged[key_vars].pct_change(12) * 100
df_yoy.columns = [f"{c}_yoy" for c in df_yoy.columns]
corr_yoy = df_yoy.corr()

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(corr_yoy, annot=True, fmt='.3f', cmap='RdYlGn', center=0,
            vmin=-1, vmax=1, square=True, linewidths=0.5,
            cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Matriz de Correlação - Variações YoY (%)', 
             fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# ============================================================================
# 4. SCATTER PLOTS COM LINHA DE TENDÊNCIA
# ============================================================================
print("\n" + "="*70)
print("4. SCATTER PLOTS - UREIA vs VARIÁVEIS CHAVE")
print("="*70)

scatter_vars = ['natural_gas_usd', 'crude_oil_usd', 'usdbrl', 'gscpi']
scatter_vars = [v for v in scatter_vars if v in df_merged.columns]

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for idx, var in enumerate(scatter_vars):
    ax = axes[idx]
    
    # Dados válidos
    data = df_merged[['urea_usd', var]].dropna()
    x = data[var].values
    y = data['urea_usd'].values
    
    # Scatter
    ax.scatter(x, y, alpha=0.5, s=20, edgecolors='k', linewidths=0.5)
    
    # Linha de tendência
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    x_line = np.linspace(x.min(), x.max(), 100)
    ax.plot(x_line, p(x_line), "r--", linewidth=2, label=f'y={z[0]:.2f}x+{z[1]:.2f}')
    
    # Correlação
    corr, pval = pearsonr(x, y)
    
    ax.set_xlabel(translations.get(var, var), fontsize=11)
    ax.set_ylabel('Ureia (US$/t)', fontsize=11)
    ax.set_title(f'Ureia vs {translations.get(var, var)}\nCorr={corr:.3f}, p={pval:.4f}',
                 fontsize=12, fontweight='bold')
    ax.legend(loc='best', fontsize=9)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================================================
# 5. SCATTER POR REGIME (PRÉ/PÓS EVENTOS IMPORTANTES)
# ============================================================================
print("\n" + "="*70)
print("5. SCATTER POR REGIME TEMPORAL")
print("="*70)

# Definir regimes baseados em eventos importantes
regimes = {
    'Pre-COVID (< 2020)': df_merged.index < pd.Timestamp('2020-01-01'),
    'COVID (2020-2021)': (df_merged.index >= pd.Timestamp('2020-01-01')) & 
                         (df_merged.index < pd.Timestamp('2022-01-01')),
    'Guerra Ucrânia (2022+)': df_merged.index >= pd.Timestamp('2022-01-01')
}

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Ureia vs Gás Natural
ax = axes[0]
colors = ['blue', 'orange', 'red']
for idx, (regime_name, mask) in enumerate(regimes.items()):
    data = df_merged.loc[mask, ['urea_usd', 'natural_gas_usd']].dropna()
    if len(data) > 0:
        ax.scatter(data['natural_gas_usd'], data['urea_usd'], 
                  alpha=0.6, s=30, label=regime_name, color=colors[idx])

ax.set_xlabel('Gás Natural (US$/MMBtu)', fontsize=11)
ax.set_ylabel('Ureia (US$/t)', fontsize=11)
ax.set_title('Ureia vs Gás Natural - Por Regime', fontsize=13, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

# Plot 2: Ureia vs USD/BRL
ax = axes[1]
for idx, (regime_name, mask) in enumerate(regimes.items()):
    data = df_merged.loc[mask, ['urea_usd', 'usdbrl']].dropna()
    if len(data) > 0:
        ax.scatter(data['usdbrl'], data['urea_usd'], 
                  alpha=0.6, s=30, label=regime_name, color=colors[idx])

ax.set_xlabel('USD/BRL', fontsize=11)
ax.set_ylabel('Ureia (US$/t)', fontsize=11)
ax.set_title('Ureia vs Câmbio - Por Regime', fontsize=13, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================================================
# 6. CROSS-CORRELATION FUNCTION (CCF)
# ============================================================================
print("\n" + "="*70)
print("6. CROSS-CORRELATION FUNCTION (CCF)")
print("="*70)

ccf_vars = ['natural_gas_usd', 'crude_oil_usd', 'usdbrl', 'gscpi']
ccf_vars = [v for v in ccf_vars if v in df_merged.columns]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

max_lag = 12

for idx, var in enumerate(ccf_vars):
    ax = axes[idx]
    
    # Dados válidos
    data = df_merged[['urea_usd', var]].dropna()
    y = data['urea_usd'].values
    x = data[var].values
    
    # Calcular CCF
    ccf_values = ccf(y, x, adjusted=False)[:max_lag+1]
    lags = np.arange(0, max_lag+1)
    
    # Plot
    ax.stem(lags, ccf_values, basefmt=' ')
    ax.axhline(0, color='k', linestyle='-', linewidth=0.8)
    ax.axhline(1.96/np.sqrt(len(y)), color='r', linestyle='--', linewidth=1, 
               label='95% CI')
    ax.axhline(-1.96/np.sqrt(len(y)), color='r', linestyle='--', linewidth=1)
    
    ax.set_xlabel('Lag (meses)', fontsize=11)
    ax.set_ylabel('Correlação', fontsize=11)
    ax.set_title(f'CCF: Ureia vs {translations.get(var, var)}', 
                 fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='best', fontsize=9)
    ax.set_xlim(-0.5, max_lag+0.5)

plt.tight_layout()
plt.show()

# ============================================================================
# 7. LEAD-LAG PLOTS
# ============================================================================
print("\n" + "="*70)
print("7. LEAD-LAG ANALYSIS")
print("="*70)

lead_lag_vars = ['natural_gas_usd', 'crude_oil_usd']
max_leads = 6

for var in lead_lag_vars:
    if var not in df_merged.columns:
        continue
    
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))
    axes = axes.flatten()
    
    for lag in range(1, max_leads+1):
        ax = axes[lag-1]
        
        # Criar dados com lag
        df_lag = df_merged[['urea_usd', var]].copy()
        df_lag[f'{var}_lag{lag}'] = df_lag[var].shift(lag)
        df_lag = df_lag.dropna()
        
        x = df_lag[f'{var}_lag{lag}'].values
        y = df_lag['urea_usd'].values
        
        # Scatter
        ax.scatter(x, y, alpha=0.5, s=20, edgecolors='k', linewidths=0.5)
        
        # Linha de tendência
        z = np.polyfit(x, y, 1)
        p = np.poly1d(z)
        x_line = np.linspace(x.min(), x.max(), 100)
        ax.plot(x_line, p(x_line), "r--", linewidth=2)
        
        # Correlação
        corr, pval = pearsonr(x, y)
        
        ax.set_xlabel(f'{translations.get(var, var)} (t-{lag})', fontsize=10)
        ax.set_ylabel('Ureia (t)', fontsize=10)
        ax.set_title(f'Lag {lag}: Corr={corr:.3f}, p={pval:.4f}',
                     fontsize=11, fontweight='bold')
        ax.grid(True, alpha=0.3)
    
    fig.suptitle(f'Lead-Lag Analysis: {translations.get(var, var)} → Ureia',
                 fontsize=14, fontweight='bold', y=1.00)
    plt.tight_layout()
    plt.show()


### Correlações Defasadas:
---

In [ ]:
import re

def lagged_corr(target: pd.Series, feature: pd.Series, max_lag: int = 24):
    # corr(target_t, feature_{t-lag}) for lag in [-max_lag, +max_lag]
    lags = np.arange(-max_lag, max_lag + 1)
    out = []
    for lag in lags:
        shifted = feature.shift(lag)
        d = pd.concat([target, shifted], axis=1).dropna()
        out.append(d.iloc[:, 0].corr(d.iloc[:, 1]) if len(d) > 5 else np.nan)
    return pd.Series(out, index=lags, name="corr")

translations = {
    "natural_gas_usd": "Gás Natural (US$/MMBtu)",
    "crude_oil_usd": "Petróleo Bruto (US$/bbl)",
    "usdbrl": "Taxa de Câmbio USD/BRL",
    "gscpi": "Global Supply Chain Pressure Index (GSCPI)",
    "oni": "Oceanic Niño Index (ONI)",
    "gpr": "Índice de Risco Geopolítico Global (GPR)",
}

lag_features = ["natural_gas_usd", "crude_oil_usd", "usdbrl", "gscpi", "oni", "gpr"]
lag_summ = []

for feat in lag_features:
    if feat not in df_merged.columns:
        continue
    lc = lagged_corr(s_urea, df_merged[feat].astype(float), max_lag=18)
    best_lag = int(lc.abs().idxmax()) if lc.dropna().size else np.nan
    best_corr = float(lc.loc[best_lag]) if not np.isnan(best_lag) else np.nan
    lag_summ.append({"feature": feat, "best_lag_months": best_lag, "corr_at_best_lag": best_corr})

    fig = plt.figure(figsize=(10.5, 4.0))
    ax = fig.add_subplot(111)
    ax.plot(lc.index, lc.values)
    ax.axhline(0, linewidth=1, color="gray", linestyle="--")
    ax.set_title(f"Correlação defasada: Ureia vs {translations.get(feat, feat)} (lag em meses)")
    ax.set_xlabel("Lag (meses). Positivo = feature atrasada; Negativo = feature adiantada")
    ax.set_ylabel("Correlação")

plt.show()

In [ ]:
def rolling_corr(a: pd.Series, b: pd.Series, window: int = 12):
    d = pd.concat([a, b], axis=1).dropna()
    return d.iloc[:, 0].rolling(window).corr(d.iloc[:, 1])

for feat in ["natural_gas_usd", "crude_oil_usd", "usdbrl", "gscpi"]:
    if feat not in df_merged.columns:
        continue
    rc = rolling_corr(s_urea, df_merged[feat].astype(float), window=24)

    fig = plt.figure(figsize=(12, 4.0))
    ax = fig.add_subplot(111)
    ax.plot(rc.index, rc.values)
    ax.axhline(0, linewidth=1)
    ax.set_title(f"Correlação móvel (24m): Ureia vs {feat}")
    ax.set_ylabel("Correlação")

In [ ]:
events = df_events.dropna(subset=["date", "event"]).copy()
events = events.sort_values("date")

# Compute event-month return
event_impact = []
for _, row in events.iterrows():
    d = row["date"]
    if d not in df_merged.index:
        continue
    
    # Usar .iloc[0] para pegar o primeiro valor se houver duplicatas
    if d in ret.index:
        ret_val = ret.loc[d]
        r = float(ret_val.iloc[0]) if isinstance(ret_val, pd.Series) else float(ret_val)
    else:
        r = np.nan
    
    if d in s_urea.index:
        price_val = s_urea.loc[d]
        p = float(price_val.iloc[0]) if isinstance(price_val, pd.Series) else float(price_val)
    else:
        p = np.nan
    
    event_impact.append({"date": d, "event": row["event"], "urea_usd_mt": p, "urea_mom_pct": r})

event_impact_df = pd.DataFrame(event_impact).sort_values("date")

# Plot with markers for events
fig = plt.figure(figsize=(12, 4.5))
ax = fig.add_subplot(111)
ax.plot(s_urea.index, s_urea.values, label="urea_usd", linewidth=1.5)

# Obter valores de ureia para as datas dos eventos
ev_dates = event_impact_df["date"].values
ev_vals = event_impact_df["urea_usd_mt"].values

# Plotar marcadores apenas onde existem valores válidos
valid_mask = ~np.isnan(ev_vals)
ax.scatter(ev_dates[valid_mask], ev_vals[valid_mask], 
           label="Eventos históricos", s=50, color='red', 
           marker='o', zorder=5, alpha=0.7)

ax.set_title("Ureia com marcação de meses com eventos históricos (main_events)", 
             fontsize=13, fontweight='bold')
ax.set_ylabel("US$/t", fontsize=11)
ax.set_xlabel("Data", fontsize=11)
ax.legend(loc="best", fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()